# mySQL Data Import
This notebook gives instructions and populates a mySQL database with the EDDB data.
## Instructions
1. Create the mySQL database. To be the same as the notebook, set the password to 917177850.
2. Change the max_packet_size to a larger number (1G)
    * In the mySQL workbench, access the database. On the left-hand side click on the "Options File" tab.
    * Click on the "Networking" tab on the ribbon.
    * Change the max_packet_size to a higher number (1G should work well).
3. Run the notebook to populate the database.

In [ ]:
# Install package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install mysql-connector-python
import mysql.connector
import json
import csv
from tqdm import tqdm

In [ ]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="917177850"
)

print(db)

cursor = db.cursor()

<h1>Create EDDB Database</h1>

In [ ]:
cursor.execute("CREATE DATABASE EDDB")

cursor.execute("SHOW DATABASES")

for x in cursor:
  print(x)

In [ ]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="917177850",
  database="EDDB"  
)

print(db)

cursor = db.cursor()

<h1>Table Creation Statements</h1>

In [ ]:
# SYSTEMS TABLE
create_statement = """
CREATE TABLE system_populated (
id INT,
name VARCHAR(255),
x FLOAT,
y FLOAT,
z FLOAT,
population BIGINT,
is_populated TINYINT,
needs_permit TINYINT,
controlling_minor_faction_id INT,
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [ ]:
# COMMODITY CATEGORY TABLE
create_statement = """
CREATE TABLE category (
id INT,
name VARCHAR(255),
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [ ]:
# COMMODITY TABLE
create_statement = """
CREATE TABLE commodity (
id INT,
name VARCHAR(255),
category_id INT,
average_price INT,
max_buy_price INT,
max_sell_price INT,
min_buy_price INT,
min_sell_price INT,
buy_price_lower_average INT,
sell_price_upper_average INT,
PRIMARY KEY (id),
FOREIGN KEY (category_id) REFERENCES category(id)
)
"""

cursor.execute(create_statement)

In [ ]:
# STATION TABLE
create_statement = """
CREATE TABLE station (
id INT,
name VARCHAR(255),
system_id INT,
max_landing_pad_size VARCHAR(255),
distance_to_star INT,
has_blackmarket TINYINT,
has_market TINYINT,
has_refuel TINYINT,
has_repair TINYINT,
has_rearm TINYINT,
has_outfitting TINYINT,
has_shipyard TINYINT,
has_docking TINYINT,
has_commodities TINYINT,
is_planetary TINYINT,
controlling_minor_faction_id INT,
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [ ]:
# LISTINGS TABLE
create_statement = """
CREATE TABLE listing (
id INT,
commodity_id INT,
station_id INT,
supply INT,
buy_price INT,
sell_price INT,
demand INT,
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [ ]:
# FACTION TABLE
create_statement = """
CREATE TABLE faction (
id INT,
name VARCHAR(255),
government VARCHAR(255),
allegiance VARCHAR(255),
home_system_id INT,
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [ ]:
# MODULE GROUP TABLE
create_statement = """
CREATE TABLE _group (
id INT,
name VARCHAR(255),
category VARCHAR(255),
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [ ]:
# MODULE TABLE
create_statement = """
CREATE TABLE module (
id INT,
group_id INT,
class INT,
rating VARCHAR(255),
price INT,
weapon_mode VARCHAR(255),
missile_type VARCHAR(255),
name VARCHAR(255),
ed_symbol VARCHAR(255),
ship VARCHAR(255),
PRIMARY KEY (id),
FOREIGN KEY (group_id) REFERENCES _group(id)
)
"""

cursor.execute(create_statement)

In [ ]:
# SYSTEMS MINOR FACTION PRESENCES
create_statement = """
CREATE TABLE minor_faction_presence (
system_id INT,
minor_faction_id INT,
influence FLOAT,
PRIMARY KEY (system_id, minor_faction_id),
FOREIGN KEY (minor_faction_id) REFERENCES faction(id),
FOREIGN KEY (system_id) REFERENCES system_populated(id)
)
"""

cursor.execute(create_statement)

In [ ]:
# STATION IMPORTS TABLE
create_statement = """
CREATE TABLE imports (
station_id INT,
commodity_id INT,
PRIMARY KEY (station_id, commodity_id),
FOREIGN KEY (station_id) REFERENCES station(id),
FOREIGN KEY (commodity_id) REFERENCES commodity(id)
)
"""

cursor.execute(create_statement)

In [ ]:
# STATION EXPORTS TABLE
create_statement = """
CREATE TABLE exports (
station_id INT,
commodity_id INT,
PRIMARY KEY (station_id, commodity_id),
FOREIGN KEY (station_id) REFERENCES station(id),
FOREIGN KEY (commodity_id) REFERENCES commodity(id)
)
"""

cursor.execute(create_statement)

In [ ]:
# STATION PROHIBITED TABLE
create_statement = """
CREATE TABLE prohibited (
station_id INT,
commodity_id INT,
PRIMARY KEY (station_id, commodity_id),
FOREIGN KEY (station_id) REFERENCES station(id),
FOREIGN KEY (commodity_id) REFERENCES commodity(id)
)
"""

cursor.execute(create_statement)

In [ ]:
# STATION SELLING MODULES TABLE
create_statement = """
CREATE TABLE selling_module (
station_id INT,
module_id INT,
PRIMARY KEY (station_id, module_id),
FOREIGN KEY (module_id) REFERENCES module(id),
FOREIGN KEY (station_id) REFERENCES station(id)
)
"""

cursor.execute(create_statement)

<h1>Add Foreign key contraints</h1>

In [ ]:
# # TWO WAY SYSTEM FACTION KEYS
# key_statement = """
# ALTER TABLE system_populated
# ADD FOREIGN KEY (controlling_minor_faction_id) REFERENCES faction(id)
# """

# cursor.execute(key_statement)

# key_statement = """
# ALTER TABLE faction
# ADD FOREIGN KEY (home_system_id) REFERENCES system_populated(id)
# """

# cursor.execute(key_statement)

<h1>Data Insert</h1>
<h1>Commodities.json</h1>

In [ ]:
file = "../data/commodities.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
category_ids = set()

for x in tqdm(data, desc="Importing Category data"):
    if not x['category']['id'] in category_ids:
        category_ids.add(x['category']['id'])
        insert_statement = """
        INSERT INTO `category` (`id`, `name`) 
        VALUES ({},'{}')
        """.format(x['category']['id'], x['category']['name'])
        cursor.execute(insert_statement)
        db.commit()

for x in tqdm(data, desc="Importing commodity data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = 'NULL'

    insert_statement = """
    INSERT INTO `commodity` (`id`, `name`, `category_id`, `average_price`, `max_buy_price`, `max_sell_price`, `min_buy_price`, `min_sell_price`, `buy_price_lower_average`, `sell_price_upper_average`) 
    VALUES ({},"{}",{},{},{},{},{},{},{},{})
    """.format(x['id'],x['name'],x['category_id'],x['average_price'],x['max_buy_price'],x['max_sell_price'],x['min_buy_price'],x['min_sell_price'],x['buy_price_lower_average'],x['sell_price_upper_average'])
    cursor.execute(insert_statement)
    db.commit()
    


<h1>factions.json</h1>

In [ ]:
file = "../data/factions.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
val = []
    
for x in tqdm(data, desc="Formatting Faction data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = -1
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    val.append((x['id'], x['name'], x['government'], x['allegiance'], x['home_system_id']))
sql = "INSERT INTO `faction` (`id`, `name`, `government`, `allegiance`, `home_system_id`) VALUES (%s,%s,%s,%s,%s)"
cursor.executemany(sql, val)    
db.commit()
print("Commited")
    


In [ ]:
file = "../data/modules.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
val = []
group_ids = set()

for x in tqdm(data, desc="Formatting group data"):
    if x['group']['id'] not in group_ids:
        group_ids.add(x['group']['id'])
        val.append((x['group']['id'], x['group']['name'], x['group']['category']))

sql = "INSERT INTO `_group` (`id`, `name`, `category`) VALUES (%s,%s,%s)"

cursor.executemany(sql, val)    
db.commit()

val = []
    
for x in tqdm(data, desc="Formatting module data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = 0
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    val.append((x['id'], x['group_id'], x['class'], x['rating'], x['price'], x['weapon_mode'], x['missile_type'], x['name'], x['ed_symbol'], x['ship']))

sql = "INSERT INTO `module` (`id`, `group_id`, `class`, `rating`, `price`, `weapon_mode`, `missile_type`, `name`, `ed_symbol`,`ship`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql, val)    
db.commit()
print("Commited")

In [ ]:
file = "../data/systems_populated.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
val = []
    
for x in tqdm(data, desc="Formatting system data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = -1
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    val.append((x['id'], x['name'], x['x'], x['y'], x['z'], x['population'], x['is_populated'], x['needs_permit'], x['controlling_minor_faction_id']))

sql = "INSERT INTO `system_populated` (`id`, `name`, `x`, `y`, `z`, `population`, `is_populated`, `needs_permit`,`controlling_minor_faction_id`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql, val)    
db.commit()
print("Commited")

In [ ]:
file = "../data/stations.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
# Need a commodity name to id map because of how the data is structured
commodity_map = {}

map_query = "SELECT id, name FROM commodity"
cursor.execute(map_query)
results = cursor.fetchall()

for commodity in results:
    commodity_map[commodity[1]] = commodity[0]
    
# STATION
val = []
imports = []
exports = []
prohibited = []
selling_modules = []
    
for x in tqdm(data, desc="Formatting station data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = -1
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    imports.append((x['id'], x['import_commodities']))
    exports.append((x['id'], x['export_commodities']))
    prohibited.append((x['id'], x['prohibited_commodities']))
    selling_modules.append((x['id'], x['selling_modules']))
    val.append((x['id'], x['name'], x['system_id'], x['max_landing_pad_size'], x['distance_to_star'], x['has_blackmarket'], x['has_market'], x['has_refuel'], x['has_repair'], x['has_rearm'], x['has_outfitting'], x['has_shipyard'], x['has_docking'], x['has_commodities'], x['is_planetary'], x['controlling_minor_faction_id']))

sql = "INSERT INTO `station` (`id`, `name`, `system_id`, `max_landing_pad_size`, `distance_to_star`, `has_blackmarket`, `has_market`, `has_refuel`, `has_repair`, `has_rearm`, `has_outfitting`, `has_shipyard`, `has_docking`, `has_commodities`, `is_planetary`,`controlling_minor_faction_id`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql, val)    
db.commit()

# IMPORTS
val = []

for x in tqdm(imports, desc="Formatting imports"):
    for commodity in x[1]:
        val.append((x[0], commodity_map[commodity]))
        
sql = "INSERT INTO `imports` (`station_id`, `commodity_id`) VALUES (%s, %s)"
cursor.executemany(sql, val)    
db.commit()  

#EXPORTS
val = []

for x in tqdm(exports, desc="Formatting exports"):
    for commodity in x[1]:
        val.append((x[0], commodity_map[commodity]))
        
sql = "INSERT INTO `exports` (`station_id`, `commodity_id`) VALUES (%s, %s)"
cursor.executemany(sql, val)    
db.commit()

#PROHIBITED
val = []

for x in tqdm(prohibited, desc="Formatting prohibited"):
    for commodity in x[1]:
        val.append((x[0], commodity_map[commodity]))
        
sql = "INSERT INTO `prohibited` (`station_id`, `commodity_id`) VALUES (%s, %s)"
cursor.executemany(sql, val)    
db.commit()

# SELLING MODULES
val = []

for x in tqdm(selling_modules, desc="Formatting selling modules"):
    for module_id in x[1]:
        val.append((x[0], module_id))
        
sql = "INSERT INTO `selling_module` (`station_id`, `module_id`) VALUES (%s, %s)"

def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in tqdm(range(0, len(lst), n), desc="Chunking..."):
        yield lst[i:i + n]

chunks = list(chunk(val, 10000))
        
for x in tqdm(chunks, desc="Uploading chunked modules data"):
    cursor.executemany(sql, x) 
    db.commit()
    
print("Commited")


In [ ]:
# Function to convert a CSV to JSON
def make_json(data, csvFilePath): 
      
    # Open a csv reader called DictReader 
    with open(csvFilePath, encoding='utf-8') as csvf: 
        csvReader = csv.DictReader(csvf) 
          
        # Convert each row into a dictionary  
        # and add it to data 
        for rows in tqdm(csvReader, desc="Reading file"): 
              
            # Assuming a column named 'No' to 
            # be the primary key 
            key = rows['id'] 
            data[key] = rows

# import listing data that is in csv file
    
chunk_size = 1000

data_dict = {}
make_json(data_dict, '../data/listings.csv')


data = []
for key in tqdm(data_dict, desc="Converting data to correct list of JSON..."):
    data.append(data_dict[key])
    
val = []
    
for x in tqdm(data, desc="Formatting listing data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = -1
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
        if f"{x[y]}" == "":
            x[y] = 0
    val.append((x['id'], x['commodity_id'], x['station_id'], x['supply'], x['buy_price'], x['sell_price'], x['demand']))

sql = "INSERT INTO `listing` (`id`, `commodity_id`, `station_id`, `supply`, `buy_price`, `sell_price`, `demand`) VALUES (%s,%s,%s,%s,%s,%s,%s)"
            
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in tqdm(range(0, len(lst), n), desc="Chunking..."):
        yield lst[i:i + n]

chunks = list(chunk(val, 10000))
        
for chunk in tqdm(chunks, desc="Uploading Chunks"):
    cursor.executemany(sql, chunk)
    
db.commit()
print("Commited")